In [1]:
from scipy.io import loadmat
import numpy as np
import seaborn as sns
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
import tensorflow as tf
from sklearn.metrics import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

import sys
sys.path.append("..") # Adds higher directory to python modules path.
from data.preprocess import *
data = preprocess(path='../data/raw/CD_PD.mat')

# Splitting the data into test/train

In [2]:
from data.preprocess import split_by_patient
# def split_by_patient(data):
    
#     bound = int(0.8*len(data))
#     idx_train = np.random.choice(len(data), bound , replace=False)
#     idx_test = np.array([e for e in np.arange(len(data)) if e not in idx_train])
# #     print(idx_train, idx_test)
#     return data[idx_train], data[idx_test]

# Evaluate function

In [5]:
from evaluation.benchmark import *
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.metrics import *
# from .evaluate import evaluate

# def run_benchmark(get_split_data,
#               get_model,
#               data,
#               k=1, 
#               model_name='Model',
#               verbose=False,
#               figsize=(15,15)
#              ):
    
#     '''
#     This function will run performance benchmark on the given model.
#     Input:
#         get_split_data : function to return sampled test train data
#         get_model : function to return a model to run benchmark on
#         data : data numpy
#         k : iteration to run the tests (higher k ~ better avg. value)
#         model_name (default='Model') : Name for the model, used in plotting ROC Curves
#         verbose (default=False) : If true, then display benchmarking result/logs in each iteration
        
#     Output:
#         models : list of models which were generated & trained during the benchmarking
#     '''
    
#     acc, rec, prec, f1 = [], [], [], []
#     fp, tp, thresh = [], [], []
#     mean_tp, _tp = [], []
#     domain = np.linspace(0, 1, 100)
#     aucs = []
#     models = []
    
#     for i in range(k):
#         if verbose:
#             print('ITER:',i)
#         x_test, x_train, y_test, y_train = get_split_data(data) #data should randomly shuffle
    
#         #train & predict
#         model = get_model()
#         model.fit(x_train, y_train, epochs=100,verbose=0)
#         models.append(model) #store model
#         y_pred = model.predict(x_test)

#         #metric eval for thresh = 0.5
#         y2 = np.array(y_pred)
#         y2[y2 >= 0.5] = 1
#         y2[y2 < 0.5]  = 0
#         t = evaluate(y_test, y2,verbose=verbose)

#         #append the evaluation results to log
#         acc.append(t[0])
#         rec.append(t[1])
#         prec.append(t[2])
#         f1.append(t[3])

#         #ROC for other thresh
#         f, t, p = roc_curve(y_test, y_pred)
#         _tp.append(np.interp(domain, f, t))

#         #AUC
#         a = auc(f, t)

#         #logging
#         fp.append(f)
#         tp.append(t)
#         thresh.append(p)
#         aucs.append(a)
        
#     if verbose:
#         print('-----thresh=0.5--------')
#         print('mean acc:',np.mean(acc))
#         print('mean recall:',np.mean(rec))
#         print('mean precision:',np.mean(prec))
#         print('mean f1:',np.mean(f1))
#         print('-----------------------')
#         plt.title('Evaluation for 10 runs (thresh=0.5)')
#         plt.plot(np.arange(k), acc, label='acc')
#         plt.plot(np.arange(k), rec, label='recall')
#         plt.plot(np.arange(k), prec,label='precision')
#         plt.plot(np.arange(k), f1,  label='f1')
#         plt.xlabel('iteration')
#         plt.ylabel('score')
#         plt.legend()
#         plt.show()
        
#     #Calculate mean and std of roc curve
#     mean_tp = np.mean(_tp, axis=0)
#     std_tp = np.std(_tp, axis=0)
#     upper_tp = mean_tp + std_tp
#     lower_tp = mean_tp - std_tp
#     mean_auc = auc(domain, mean_tp)
    
#     #ref: https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html#sphx-glr-auto-examples-model-selection-plot-roc-crossval-py
#     plt.figure(figsize=figsize)
#     plt.title('ROC Curve')
#     plt.xlabel('fpr')
#     plt.ylabel('tpr')
#     plt.plot([0,1],[0,1],label='Chance', linestyle='dashed',color='red')
    
#     #plot ROC Curve
#     for i in range(k):
#         plt.plot(fp[i], tp[i], label='%s iteration:%d AUC=%.2f' % (model_name,i,aucs[i]), alpha=0.2)
#     plt.plot(domain, mean_tp, color='blue', alpha=1, label='average AUC=%.2f' % mean_auc)
#     # plt.plot(domain, upper_tp, color='gray', alpha=0.1)
#     # plt.plot(domain, lower_tp, color='gray', alpha=0.1)
#     plt.fill_between(domain, upper_tp, lower_tp, color='grey', alpha=.2, label=r'$\pm$ 1 std. dev.')
#     plt.legend()
#     plt.show()
    
#     return models



# Random Baseline

This baseline will uniformly random the label (0 = No, 1 = Yes), and used to compare with other models. Other models should exceed this baseline.

In [6]:
data_train, data_test = split_by_patient(data)
y_true = data_test['diagnosis'].astype(np.float32)
y_pred = np.array([np.random.choice(2) for i in range(len(y_true))])
acc, rec, prec, f1 = evaluate(y_true, y_pred, verbose=True)

--------evaluation---------
accuracy: 0.625
recall: 0.625
precision: 0.7142857142857143
f1 score: 0.6666666666666666
---------------------------


# All One Baseline

This baseline will predict y = 1 for any x.

In [17]:
data_train, data_test = split_by_patient(data)
y_true = data_test['diagnosis'].astype(np.float32)
y_pred = np.ones(shape=(y_true.shape))
acc, rec, prec, f1 = evaluate(y_true, y_pred, verbose=True)

--------evaluation---------
accuracy: 0.675
recall: 1.0
precision: 0.675
f1 score: 0.8059701492537313
---------------------------


# All Zero Baseline

This baseline will predict y = 0 for any x.

In [20]:
data_train, data_test = split_by_patient(data)
y_true = data_test['diagnosis'].astype(np.float32)
y_pred = np.zeros(shape=(y_true.shape))
acc, rec, prec, f1 = evaluate(y_true, y_pred, verbose=True)

--------evaluation---------
accuracy: 0.35
recall: 0.0
precision: 0.0
f1 score: 0.0
---------------------------


# Performance Evaluation

We will run the model training & inference k times and evaluate the average performance 

In [ ]:
from evaluate.benchmark import run_benchmark
# def run_benchmark(get_split_data,
#               get_model,
#               data,
#               k=1, 
#               model_name='Model',
#               verbose=False
#              ):
    
#     '''
#     This function will run performance benchmark on the given model.
#     Input:
#         get_split_data : function to return sampled test train data
#         get_model : function to return a model to run benchmark on
#         k : iteration to run the tests (higher k ~ better avg. value)
#         model_name (default='Model') : Name for the model, used in plotting ROC Curves
#         verbose (default=False) : If true, then display benchmarking result/logs in each iteration
        
#     Output:
#         models : list of models which were generated & trained during the benchmarking
#     '''
    
#     acc, rec, prec, f1 = [], [], [], []
#     fp, tp, thresh = [], [], []
#     mean_tp, _tp = [], []
#     domain = np.linspace(0, 1, 100)
#     aucs = []
#     models = []
    
#     for i in range(k):
#         if verbose:
#             print('ITER:',i)
#         x_test, x_train, y_test, y_train = get_split_data(data) #data should randomly shuffle
    
#         #train & predict
#         model = get_model()
#         model.fit(x_train, y_train, epochs=100,verbose=0)
#         models.append(model) #store model
#         y_pred = model.predict(x_test)

#         #metric eval for thresh = 0.5
#         y2 = np.array(y_pred)
#         y2[y2 >= 0.5] = 1
#         y2[y2 < 0.5]  = 0
#         t = evaluate(y_test, y2,verbose=verbose)

#         #append the evaluation results to log
#         acc.append(t[0])
#         rec.append(t[1])
#         prec.append(t[2])
#         f1.append(t[3])

#         #ROC for other thresh
#         f, t, p = roc_curve(y_test, y_pred)
#         _tp.append(np.interp(domain, f, t))

#         #AUC
#         a = auc(f, t)

#         #logging
#         fp.append(f)
#         tp.append(t)
#         thresh.append(p)
#         aucs.append(a)
        
#     if verbose:
#         print('-----thresh=0.5--------')
#         print('mean acc:',np.mean(acc))
#         print('mean recall:',np.mean(rec))
#         print('mean precision:',np.mean(prec))
#         print('mean f1:',np.mean(f1))
#         print('-----------------------')
#         plt.title('Evaluation for 10 runs (thresh=0.5)')
#         plt.plot(np.arange(k), acc, label='acc')
#         plt.plot(np.arange(k), rec, label='recall')
#         plt.plot(np.arange(k), prec,label='precision')
#         plt.plot(np.arange(k), f1,  label='f1')
#         plt.xlabel('iteration')
#         plt.ylabel('score')
#         plt.legend()
#         plt.show()
        
#     #Calculate mean and std of roc curve
#     mean_tp = np.mean(_tp, axis=0)
#     std_tp = np.std(_tp, axis=0)
#     upper_tp = mean_tp + std_tp
#     lower_tp = mean_tp - std_tp
#     mean_auc = auc(domain, mean_tp)
    
#     #ref: https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html#sphx-glr-auto-examples-model-selection-plot-roc-crossval-py
#     plt.figure(figsize=(15,15))
#     plt.title('ROC Curve')
#     plt.xlabel('fpr')
#     plt.ylabel('tpr')
#     plt.plot([0,1],[0,1],label='Chance', linestyle='dashed',color='red')
    
#     #plot ROC Curve
#     for i in range(k):
#         plt.plot(fp[i], tp[i], label='%s iteration:%d AUC=%.2f' % (model_name,i,aucs[i]), alpha=0.2)
#     plt.plot(domain, mean_tp, color='blue', alpha=1, label='average AUC=%.2f' % mean_auc)
#     # plt.plot(domain, upper_tp, color='gray', alpha=0.1)
#     # plt.plot(domain, lower_tp, color='gray', alpha=0.1)
#     plt.fill_between(domain, upper_tp, lower_tp, color='grey', alpha=.2, label=r'$\pm$ 1 std. dev.')
#     plt.legend()
#     plt.show()
    
#     return models

